# Simulator code
A program to simulate quantum circuits in Python. It lets you:

1. Create a quantum circuit in ground state
2. Apply `X`, `H` and `CX` quantum gates
3. Measure the final circuit state

## Bonus tasks
Both bonus tasks have been implemented:

1. Parametric gates
2. Global params

To run these, look at the driver code and uncomment the appropriate parts.

In [5]:
import numpy as np
import math
import random

gates = {
  "x": np.array([
        [0, 1],
        [1, 0]
      ]),
  "h": 1/math.sqrt(2) * np.array([
        [1, 1],
        [1, -1]
      ]),
  "cx": np.array([
        [1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 0, 1],
        [0, 0, 1, 0],
      ])      
}


def get_u3(params, global_params):
    """
    Return u3 vector using provided params

    :param params: dict containing theta, phi and lambda values
    :param global_params: dict containing global_1 and global_2
    :return: vector of size 2 x 2 containing values of u3 gate
    """ 
    
    theta = params["theta"]
    if theta == "global_1":
      theta = global_params["global_1"]

    phi = params["phi"]
    if phi == "global_2":
      phi = global_params["global_2"]

    lbda = params["lambda"]

    u3 = np.array([
                  [np.cos(theta/2), -np.exp(1j * lbda)*np.sin(theta/2)],
                  [np.exp(1j * phi)*np.sin(theta/2), np.exp(1j * (lbda + phi))*np.cos(theta/2)]
    ])

    return u3

def get_ground_state(num_qubits):
    """
    Return ground state vector

    Vector will have all zeroes except first element which is 1

    :param num_qubits: Number of qubits in circuit
    :return: vector of size 2**num_qubits
    """ 

    state = np.full((2**num_qubits), 0)
    state[0] = 1
    return state

def get_operator(total_qubits, gate_unitary, target_qubits):
    """
    Get matrix operator for quantum gate

    Operator depends on total qubits and qubits to be targeted.

    :param total_qubits: Number of qubits in circuit
    :param gate_unitary: Unitary gate of 2 x 2 size
    :param target_qubits: Qubits to operate upon
    :return: vector of size 2**total_qubits x 2**total_qubits
    """ 
 
    operator = 1
    I = np.identity(2) # Unitary operators are 2*2 matrices
    
    for qubit in target_qubits:
      # tensor product of 'total_qubits' with 'gate_unitary' at 'qubit' position
      k_prefix = 1
      for prefix_position in range(0, qubit):
        k_prefix = np.kron(k_prefix, I)
      
      k_suffix = 1
      for suffix_position in range(qubit+1, total_qubits):
        k_suffix = np.kron(k_suffix, I)
      
      qbit_operator=np.kron(np.kron(k_prefix, gate_unitary), k_suffix)
      operator = np.dot(operator, qbit_operator)

    return operator

def run_program(initial_state, program, global_params):
    """
    Run program on circuit and return final state

    For each gate, calculate matrix operator, perform
    multiplication and return result

    :param initial_state: Ground state of circuit
    :param program: Dictionary of gate name and target qbits
    :param global_params: Dictionary containing values to find parametric gate
    :return: final state
    """

    state = initial_state
    total_qubits = int(math.sqrt(state.shape[0]))

    for operation in program:
      gate_name = operation["gate"]
      # get matrix for gate name
      if gate_name == "u3":
        gate = get_u3(operation["params"], global_params)
      else:
        gate = gates[gate_name]
      
      target_qubits = operation["target"]

      if operation["gate"] == "cx":
        # hack: CX gate can directly act on a 2 Qubit circuit
        # only works for 2 qubit circuits
        operator = gate
      else:
        operator = get_operator(total_qubits, gate, target_qubits)
      
      state = np.dot(state, operator)
      
    return state


def measure_all(state_vector):
    """
    Measure classical state of quantum circut

    Random value is returned based on probability
    distribution of state vector

    :param state_vector: Final quantum state of circuit
    :return: measured classical state of circuit in decimal format
    """

    probabilities = np.square(state_vector) # Find probabilities using amplitudes
    
    # Find single random value using probability distribution
    measured_state = random.choices(range(0, len(state_vector)), probabilities)
    return measured_state[0]

def get_counts(state_vector, num_shots):
    """
    Observe classical state multiple times and return the distribution

    :param state_vector: Final quantum state of circuit
    :param num_shots: Number of times the state is measured
    :return: Distribution of measured classical states in binary format
    """

    # Initialize with 0 count for each state
    result = {
      x: 0 for x in range(0, len(state_vector))
    }

    # Get results for 'num_shots' number of trials
    for test_case in range(0, num_shots):
      measurement = measure_all(state_vector)
      result[measurement] =  result[measurement] + 1
    
    # remove states with 0 occurances
    result = {state: count for state, count in result.items() if count != 0}

    # find binary places required to represent circuit state
    last_item = len(state_vector) - 1
    last_item_binary = bin(last_item)
    places = len(last_item_binary) - 2 # Remove 0b

    # convert states to binary format
    formatting_string = "{0:0" + str(places) + "b}"
    formatted_result = {"{0:02b}".format(state): count for state, count in result.items()}

    return formatted_result


# Driver code

In [6]:
# Define program:

my_circuit = [
    { "gate": "h", "target": [0] }, 
    { "gate": "cx", "target": [0, 1] },

    # for parametric gates
    # { "gate": "u3", "params": { "theta": 3.1415, "phi": 1.5708, "lambda": -3.1415 }, "target": [0] },

    # for global parameters
    # { "gate": "u3", "params": { "theta": "global_1", "phi": "global_2", "lambda": -3.1415 }, "target": [0] }
]

# Create "quantum computer" with 2 qubits (this is actually just a vector :) )

my_qpu = get_ground_state(2)

# Run circuit
global_params = { "global_1": 3.1415, "global_2": 1.5708 }
final_state = run_program(my_qpu, my_circuit, global_params)
print("final state", final_state)

# Read results

counts = get_counts(final_state, 1000)

print("measured counts", counts)

# Should print something like:
# {
#   "00": 502,
#   "11": 498
# }

# Voila!


final state [0.70710678 0.         0.         0.70710678]
measured counts {'00': 506, '11': 494}
